In [1]:
import gzip
from collections import defaultdict
import math
import scipy.optimize
from sklearn import svm
import numpy as np
import string
import random
import string
from sklearn import linear_model

In [2]:
def readGz(path):
    for l in gzip.open(path, 'rt'):
        yield eval(l)

In [3]:
def readCSV(path):
    f = gzip.open(path, 'rt')
    f.readline()
    for l in f:
        u,b,r = l.strip().split(',')
        r = int(r)
        yield u,b,r

In [4]:
allRatings = []
for l in readCSV("train_Interactions.csv.gz"):
    allRatings.append(l)

In [113]:
totalRead = 190000
ratingsTrain = allRatings[:totalRead]
ratingsValid = allRatings[totalRead:]
ratingsPerUser = defaultdict(list)
ratingsPerItem = defaultdict(list)
popularity = defaultdict(int)
for u,b,r in ratingsTrain:
    ratingsPerUser[u].append((b,r))
    ratingsPerItem[b].append((u,r))
    popularity[b] += 1

In [114]:
userSet = set()
bookSet = set()
readSet = set()

for u,b,r in allRatings:
    userSet.add(u)
    bookSet.add(b)
    readSet.add((u,b))

lUserSet = list(userSet)
lBookSet = list(bookSet)

notRead = set()
for u,b,r in ratingsValid:
    #u = random.choice(lUserSet)
    b = random.choice(lBookSet)
    if (u,b) in readSet or (u,b) in notRead:
        continue
    notRead.add((u,b))
    
# count = 0
# while count < 5000:
#     count += 1
#     u = random.choice(lUserSet)
#     b = random.choice(lBookSet)
#     while (u,b) in readSet or (u,b) in notRead:
#         b = random.choice(lBookSet)
#     notRead.add((u, b))

readValid = set()
for u,b,r in ratingsValid:
    readValid.add((u,b))

In [ ]:
sortedBook = sorted(popularity.items(), key=lambda x: x[1], reverse = True)

In [7]:
def Jaccard(s1, s2):
    numer = len(s1.intersection(s2))
    denom = len(s1.union(s2))
    if denom > 0:
        return numer/denom
    return 0

In [8]:
def Cosine(s1, s2):
    s1 = list(s1)
    s2 = list(s2)
    if (len(s1) == 0 or len(s2) == 0):
        return 0
    nom = 0
    s1_book = [i[0] for i in s1]
    s2_book = [i[0] for i in s2]
    s1_rate = [i[1] for i in s1]
    s2_rate = [i[1] for i in s2]
    for b,r in s1:
        if b in s2_book:
            nom += s1_rate[s1_book == b] * s2_rate[s2_book == b]
    den = np.sqrt(np.sum(np.square(s1_rate))) * np.sqrt(np.sum(np.square(s2_rate)))
    if den != 0:
        return nom / den
    return 0

In [9]:
def Perlson(s1, s2):
    s1 = list(s1)
    s2 = list(s2)
    if (len(s1) == 0 or len(s2) == 0):
        return 0
    nom = 0
    s1_book = [i[0] for i in s1]
    s2_book = [i[0] for i in s2]
    s1_rate = [i[1] for i in s1]
    s2_rate = [i[1] for i in s2]
    s1_avg = np.sum(s1_rate) / len(s1)
    s2_avg = np.sum(s2_rate) / len(s2)
    s1_rate = [i - s1_avg for i in s1_rate]
    s1_rate = [i - s2_avg for i in s2_rate]
    for b,r in s1:
        if b in s2_book:
            nom += s1_rate[s1_book == b] * s2_rate[s2_book == b]
    den = np.sqrt(np.sum(np.square(s1_rate))) * np.sqrt(np.sum(np.square(s2_rate)))
    if den != 0:
        return nom / den
    return 0

In [10]:
def maxSimUser(u, b):
    maxSim = 0
    users = set(ratingsPerItem[b])
    for b2,_ in ratingsPerUser[u]:
        sim = Jaccard(users,set(ratingsPerItem[b2]))
        if sim > maxSim:
            maxSim = sim
    return maxSim

In [11]:
def maxSimBook(b, u):
    maxSim = 0
    books = set(ratingsPerUser[u])
    for u2,_ in ratingsPerItem[b]:
        sim = Jaccard(books,set(ratingsPerUser[u2]))
        if sim > maxSim:
            maxSim = sim
    return maxSim

In [68]:
def maxCosSimUser(u, b):
    maxCosSim = 0
    users = set(ratingsPerItem[b])
    for b2,_ in ratingsPerUser[u]:
        sim = Cosine(users,set(ratingsPerItem[b2]))
        if sim > maxCosSim:
            maxCosSim = sim
    return maxCosSim

In [69]:
def maxCosSimBook(b, u):
    maxCosSim = 0
    books = set(ratingsPerUser[u])
    for u2,_ in ratingsPerItem[b]:
        sim = Cosine(books,set(ratingsPerUser[u2]))
        if sim > maxCosSim:
            maxCosSim = sim
    return maxCosSim

In [13]:
def maxPerlSimUser(u, b):
    maxPerlSim = 0
    users = set(ratingsPerItem[b])
    for b2,_ in ratingsPerUser[u]:
        sim = Perlson(users,set(ratingsPerItem[b2]))
        if sim > maxPerlSim:
            maxPerlSim = sim
    return maxPerlSim

In [15]:
def maxPerlSimBook(b, u):
    maxPerlSim = 0
    books = set(ratingsPerUser[u])
    for u2,_ in ratingsPerItem[b]:
        sim = Perlson(books,set(ratingsPerUser[u2]))
        if sim > maxPerlSim:
            maxPerlSim = sim
    return maxPerlSim

In [ ]:
correct = 0
p0, p1 = 0,0
for (label,sample) in [(1, readValid), (0, notRead)]:
    for (u,b) in sample:
        maxSim = maxSimCal(u, b)
        pred = 0
        if maxSim > 0.015 or len(ratingsPerItem[b]) > 40:
            pred = 1
            p1 += 1
        else:
            p0 += 1
        if pred == label:
            correct += 1

In [ ]:
print(correct / 10000)

In [115]:
validData = []
for u, b in readValid:
    feature = []
    popularity = len(ratingsPerItem[b]) / len(bookSet)
    maxUser = maxSimUser(u, b)
    maxBook = maxSimBook(b, u)
    maxCosUser = maxCosSimUser(u, b)
    maxCosBook = maxCosSimBook(b, u)
    maxPerlUser = maxPerlSimUser(u, b)
    maxPerlBook = maxPerlSimBook(b, u)
    feature = [1, popularity, maxUser, maxBook, maxCosUser, maxCosBook, maxPerlUser, maxPerlBook]
    validData.append([feature, 1])
    
for u, b in notRead:
    feature = []
    popularity = len(ratingsPerItem[b]) / len(bookSet)
    maxUser = maxSimUser(u, b)
    maxBook = maxSimBook(b, u)
    maxCosUser = maxCosSimUser(u, b)
    maxCosBook = maxCosSimBook(b, u)
    maxPerlUser = maxPerlSimUser(u, b)
    maxPerlBook = maxPerlSimBook(b, u)
    feature = [1, popularity, maxUser, maxBook, maxCosUser, maxCosBook, maxPerlUser, maxPerlBook]
    validData.append([feature, 0])   

In [116]:
random.shuffle(validData)
validX = []
validY = []

for data in validData:
    validX.append(data[0])
    validY.append(data[1])

In [153]:
clf = linear_model.LogisticRegression(C=5000,solver='lbfgs').fit(validX, validY)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


In [154]:
validscore = clf.score(validX, validY)
print(validscore)

0.6789239016081359


In [155]:
predictions = open("predictions_Read.txt", 'w')
for l in open("pairs_Read.txt"):
    if l.startswith("userID"):
        predictions.write(l)
        continue
    u,b = l.strip().split('-')
    pop = len(ratingsPerItem[b]) / len(bookSet)
    maxUser = maxSimUser(u, b)
    maxBook = maxSimBook(b, u)
    maxCosUser = maxCosSimUser(u, b)
    maxCosBook = maxCosSimBook(b, u)
    maxPerlUser = maxPerlSimUser(u, b)
    maxPerlBook = maxPerlSimBook(b, u)
    feature = [1, pop, maxUser, maxBook, maxCosUser, maxCosBook, maxPerlUser, maxPerlBook]
    pred = 0
    if clf.predict([feature])[0] == 1:
        pred = 1
    _ = predictions.write(u + '-' + b + ',' + str(pred) + '\n')

predictions.close()

In [ ]:
allRatings = []
userRatings = defaultdict(list)
bookRatings = defaultdict(list)
user_book = defaultdict(list)
book_user = defaultdict(list)
valid = []

count = 0

for user,book,r in readCSV("train_Interactions.csv.gz"):
    count += 1
    r = int(r)
    if count <= 170000:
        allRatings.append(r)
        userRatings[user].append(r)
        bookRatings[book].append(r)
        user_book[user].append(book)
        book_user[book].append(user)
    else:
        l = []
        l.append(user)
        l.append(book)
        l.append(r)
        valid.append(l)

globalAverage = sum(allRatings) / len(allRatings)
userAverage = {}
for u in userRatings:
    userAverage[u] = sum(userRatings[u]) / len(userRatings[u])
userSum = {}
for u in userRatings:
    userSum[u] = sum(userRatings[u])
bookSum = {}
for b in bookRatings:
    bookSum[b] = sum(bookRatings[b])
    
alpha = globalAverage
beta_u = defaultdict(int)
beta_b = defaultdict(int)
lambda_ = 3

In [ ]:
for u in userRatings:
    beta_u[u] = 0
for b in bookRatings:
    beta_b[b] = 0
    
for i in range(300):
    sum_beta_u = 0
    sum_beta_b = 0
#     beta_u_new = defaultdict(int)
#     beta_b_new = defaultdict(int)
#     for u in userRatings:
#         beta_u_new[u] = 0
#     for b in bookRatings:
#         beta_b_new[b] = 0
    for user in user_book:
        for book in user_book[user]:
            sum_beta_u += beta_u[user]
            sum_beta_b += beta_b[book]
    alpha = (sum(allRatings) - (sum_beta_b + sum_beta_u)) / 170000
    for user in user_book:
    #         print(userRatings[user])
        sum_ratings_u = userSum[user]
    #         print(sum_ratings_u)
        numOfBook = len(userRatings[user])
#         print(numOfBook)
        sum_beta_b = 0
        for book in user_book[user]:
            sum_beta_b += beta_b[book]
        beta_u[user] = (sum_ratings_u - alpha * numOfBook - sum_beta_b) / (lambda_ + numOfBook)
    #         print(beta_u_new[user])

    for book in book_user:
        sum_ratings_b = bookSum[book]
        numOfUser = len(bookRatings[book])
#         print(numOfUser)
        sum_beta_u = 0
        for user in book_user[book]:
            sum_beta_u += beta_u[user]
        beta_b[book] = (sum_ratings_b - alpha * numOfUser - sum_beta_u) / (lambda_ + numOfUser)
        
#     alpha = alpha_new
#     beta_u = beta_u_new
#     beta_b = beta_b_new


In [ ]:
sum_err = 0
for i in valid:
    predict_rating = alpha + beta_u[i[0]] + beta_b[i[1]]
#     print(predict_rating)
    sum_err += (predict_rating - i[2]) ** 2
MSE = sum_err / len(valid)
print("mse = ", MSE)

In [ ]:
predictions = open("predictions_Rating.txt", 'w')
for l in open("pairs_Rating.txt"):
    if l.startswith("userID"):
        #header
        predictions.write(l)
        continue
    u,b = l.strip().split('-')
    if u in userRatings and b in bookRatings:
        predict_rating = alpha + beta_u[u] + beta_b[b]
        predictions.write(u + '-' + b + ',' + str(predict_rating) + '\n')
    elif u in userRatings:
        predict_rating = alpha + beta_u[u]
        predictions.write(u + '-' + b + ',' + str(predict_rating) + '\n')
    elif b in bookRatings:
        predict_rating = alpha + beta_b[b]
        predictions.write(u + '-' + b + ',' + str(predict_rating) + '\n')
    else:
        predict_rating = alpha
        predictions.write(u + '-' + b + ',' + str(predict_rating) + '\n')